In [1]:
%matplotlib wx

import numpy as np 
import rospy
import rosbag
from matplotlib import pylab as plt
plt.ion()
#scikits learn is needed to filter outliers
from sklearn import linear_model, datasets

In [2]:
#specify all the topics to use (order matters, see topics_node_to_node)
topics = ['/bbb2/0x2017_0xd','/bbb2/0x2017_0xe','/bbb2/0x2017_0xf','/bbb2/0x2017_0x10','/bbb2/0x2017_0x11','/bbb2/0x2017_0x12','/bbb2/0x2017_0x13','/bbb2/0x2017_0x14',
          '/bbb2/0x2717_0xd','/bbb2/0x2717_0xe','/bbb2/0x2717_0xf','/bbb2/0x2717_0x10','/bbb2/0x2717_0x11','/bbb2/0x2717_0x12','/bbb2/0x2717_0x13','/bbb2/0x2717_0x14',
          '/bbb4/0x2017_0xd','/bbb4/0x2017_0xe','/bbb4/0x2017_0xf','/bbb4/0x2017_0x10','/bbb4/0x2017_0x11','/bbb4/0x2017_0x12','/bbb4/0x2017_0x13','/bbb4/0x2017_0x14',
          '/bbb4/0x2717_0xd','/bbb4/0x2717_0xe','/bbb4/0x2717_0xf','/bbb4/0x2717_0x10','/bbb4/0x2717_0x11','/bbb4/0x2717_0x12','/bbb4/0x2717_0x13','/bbb4/0x2717_0x14',
          '/bbb6/0x2017_0xd','/bbb6/0x2017_0xe','/bbb6/0x2017_0xf','/bbb6/0x2017_0x10','/bbb6/0x2017_0x11','/bbb6/0x2017_0x12','/bbb6/0x2017_0x13','/bbb6/0x2017_0x14',
          '/bbb6/0x2717_0xd','/bbb6/0x2717_0xe','/bbb6/0x2717_0xf','/bbb6/0x2717_0x10','/bbb6/0x2717_0x11','/bbb6/0x2717_0x12','/bbb6/0x2717_0x13','/bbb6/0x2717_0x14',
          '/bbb8/0x2017_0xd','/bbb8/0x2017_0xe','/bbb8/0x2017_0xf','/bbb8/0x2017_0x10','/bbb8/0x2017_0x11','/bbb8/0x2017_0x12','/bbb8/0x2017_0x13','/bbb8/0x2017_0x14',
          '/bbb8/0x2717_0xd','/bbb8/0x2717_0xe','/bbb8/0x2717_0xf','/bbb8/0x2717_0x10','/bbb8/0x2717_0x11','/bbb8/0x2717_0x12','/bbb8/0x2717_0x13','/bbb8/0x2717_0x14',
          '/bbb10/0x2017_0xd','/bbb10/0x2017_0xe','/bbb10/0x2017_0xf','/bbb10/0x2017_0x10','/bbb10/0x2017_0x11','/bbb10/0x2017_0x12','/bbb10/0x2017_0x13','/bbb10/0x2017_0x14',
          '/bbb10/0x2717_0xd','/bbb10/0x2717_0xe','/bbb10/0x2717_0xf','/bbb10/0x2717_0x10','/bbb10/0x2717_0x11','/bbb10/0x2717_0x12','/bbb10/0x2717_0x13','/bbb10/0x2717_0x14',
          '/bbb12/0x2017_0xd','/bbb12/0x2017_0xe','/bbb12/0x2017_0xf','/bbb12/0x2017_0x10','/bbb12/0x2017_0x11','/bbb12/0x2017_0x12','/bbb12/0x2017_0x13','/bbb12/0x2017_0x14',
          '/bbb12/0x2717_0xd','/bbb12/0x2717_0xe','/bbb12/0x2717_0xf','/bbb12/0x2717_0x10','/bbb12/0x2717_0x11','/bbb12/0x2717_0x12','/bbb12/0x2717_0x13','/bbb12/0x2717_0x14'
         ]
#the node tuples (from-to) corresponding to the above topics
#Use actual numbers (i.e. node 1 is 1, not 0)
topics_node_to_node = np.array([[2,13],
                               [2,14],
                               [2,15],
                               [2,16],
                                [2,17],
                               [2,18],
                               [2,19],
                               [2,20],
                               [1,13],
                                [1,14],
                                [1,15],
                                [1,16],
                                [1,17],
                               [1,18],
                               [1,19],
                               [1,20],
                                
                                [4,13],
                               [4,14],
                               [4,15],
                               [4,16],
                                [4,17],
                               [4,18],
                               [4,19],
                               [4,20],
                               [3,13],
                                [3,14],
                                [3,15],
                                [3,16],
                                [3,17],
                               [3,18],
                               [3,19],
                               [3,20],
                                
                                [6,13],
                               [6,14],
                               [6,15],
                               [6,16],
                                [6,17],
                               [6,18],
                               [6,19],
                               [6,20],
                               [5,13],
                                [5,14],
                                [5,15],
                                [5,16],
                                [5,17],
                               [5,18],
                               [5,19],
                               [5,20],
                                
                                [8,13],
                               [8,14],
                               [8,15],
                               [8,16],
                                [8,17],
                               [8,18],
                               [8,19],
                               [8,20],
                               [7,13],
                                [7,14],
                                [7,15],
                                [7,16],
                                [7,17],
                               [7,18],
                               [7,19],
                               [7,20],
                                
                                [10,13],
                               [10,14],
                               [10,15],
                               [10,16],
                                [10,17],
                               [10,18],
                               [10,19],
                               [10,20],
                               [9,13],
                                [9,14],
                                [9,15],
                                [9,16],
                                [9,17],
                               [9,18],
                               [9,19],
                               [9,20],
                                
                                [12,13],
                               [12,14],
                               [12,15],
                               [12,16],
                                [12,17],
                               [12,18],
                               [12,19],
                               [12,20],
                               [11,13],
                                [11,14],
                                [11,15],
                                [11,16],
                                [11,17],
                               [11,18],
                               [11,19],
                               [11,20],
                               ],dtype=int) #corresponds to topics above


bag_names = []
measured_distances = []
node_to_sensor = []

#add bag files to process
bag_names.append("outdoor_ranging_8_anchors_round_2_2015-10-15-16-24-00.bag")

In [3]:
#process all bag files by filtering out bad data (this takes a while).
#plot raw data
#plt.close('all')
bag_means = []
bag_std = []
bag_means_raw = []
bag_stds_raw = []
bag_means_nodes = []
bag_std_nodes = []
data_clean = []
for n_i,n in enumerate(bag_names):
    print n
    bag = rosbag.Bag(n)
    data = {}
    topics_mean = np.zeros(len(topics))
    topics_std = np.zeros(len(topics))
    plt.figure()
    d_supahclean = []
    for i,t in enumerate(topics):
        plt.subplot(12,1,i/8+1)
        data = []
        for m in bag.read_messages(topics=[t]):
            data.append((m[1].header.stamp.to_sec(),m[1].data))
        d = np.array(data)
        try:
            d_clean = d[np.where((d[:,1]>3.5)&(d[:,1]<25))[0]] #remove bad measurements
        except:
            d_supahclean.append(np.zeros((1,2)))
            print "err"
            continue
        if(d_clean.shape[0]>10):
            #we use RANSAC to robustly estimate the mean and variance and filter out outliers
            model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
            model_ransac.fit(np.arange(d_clean.shape[0]).reshape((-1,1)), d_clean)
            inlier_mask = model_ransac.inlier_mask_
            outlier_mask = np.logical_not(inlier_mask)
            m = d_clean[inlier_mask,1].mean()
            std = d_clean[inlier_mask,1].std()
            d_outlier = d_clean[np.where((np.abs(d_clean[:,1]-m)<3*std))[0],:] #keep points within 3 std dev of the mean (less restrictive than RANSAC)
            topics_mean[i] = d_outlier[:,1].mean()
            topics_std[i] = d_outlier[:,1].std()
            #plt.plot(d_clean)
            #plt.plot(d_clean[inlier_mask])
            #plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1],'.')
            #plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1])
            
            d = d_outlier.copy()
            good_data = []
            good_data.append(d[0])
            v_d = np.zeros(d.shape)
            for i in xrange(1,d.shape[0]):
                t_1 = good_data[-1][0]
                t = d[i,0]
                p_1 = good_data[-1][1]
                p = d[i,1]
                v = np.abs((p-p_1)/(t-t_1))
                v_d[i]=v
                if(v<2.5):
                    good_data.append(d[i])
            good_data=np.array(good_data)
            #plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1],'+')
            plt.plot(good_data[:,0]-d_outlier[0,0],good_data[:,1],'.')
            plt.plot(good_data[:,0]-d_outlier[0,0],good_data[:,1],'-')
            d_supahclean.append(good_data.copy())
        else:
            topics_mean[i] = 0
            topics_std[i] = 0
            d_supahclean.append(np.zeros((1,2)))
    data_clean.append(d_supahclean) #contains the valid data for each topic


outdoor_ranging_8_anchors_round_2_2015-10-15-16-24-00.bag


In [4]:
#convert to fixed rate by taking the mean per time step
meas = []
for n_i,n in enumerate(bag_names):
    data = data_clean[n_i]
    t0 = np.max([d[0,0] for d in data[:8]])
    tend = np.min([d[-1,0] for d in data[:8]])
    timesteps = np.arange(t0,tend,0.25)
    measurements = np.zeros((timesteps.shape[0],len(topics)))
    
    for t_i, t in enumerate(timesteps):
        for j in xrange(len(topics)):
            d = data[j]
            #get all matching data
            idx = np.where((d[:,0]>=t) & (d[:,0]<t+0.25))[0]
            if(idx.shape>0):
                measurements[t_i,j] = np.median(d[idx,1])
            else:
                measurements[t_i,j] = 0 
    meas.append(measurements)#[np.where(np.sum((measurements[:,:8]>1) & (measurements[:,:8]<30),1)>2)]) #how many valid measurements do we need?
    #TODO: for multiple nodes, make sure each one has at least 4  ranging measurements

/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:71: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
#Plot data samples with at least 4 measurements per floating node 
p1 = plt.figure()
p2 = plt.figure()
for i in xrange(12):
    plt.figure(p1.number)
    plt.subplot(3,4,i+1)
    m = [np.where(np.sum((meas[0][:,i*8:(i+1)*8]>1) & (meas[0][:,i*8:(i+1)*8]<30),1)>3)]
    plt.plot(meas[0][:,i*8:(i+1)*8])
    plt.figure(p2.number)
    for j in xrange(8):
        plt.subplot(2,4,j+1)
        plt.plot(meas[0][:,i*8:(i+1)*8][:,j])

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:7: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:7: RuntimeWarning: invalid value encountered in less


In [6]:
#This code is used to generate all the constraints.

#Define the cost function and its derivatives (x,y) are coordinates, m are measurements and o are measurement offsets
def cost_function(x,y,m,o):
    '''
        Assumes x and y are Nx3, m and o are 1d vectors
    '''
    dist_xy = np.sqrt(np.sum((x-y)**2,1)) #1d vector
    cost_per_element = (dist_xy-(m-o))**2
    total_cost = cost_per_element.mean()
    return total_cost

def derivative_cost_function(x,y,m,o):
    '''
        Derivatives of the cost function wrt x,y and o
    '''
    dist_xy = np.sqrt(np.sum((x-y)**2,1)) #1d vector
    factor = 2*(dist_xy-(m-o))/dist_xy/m.shape[0]
    #derivative with respect to x
    deriv_x = factor.reshape((-1,1))*(x-y)
    #derivative with respect to y
    deriv_y = -deriv_x
    #derivative with respect to o
    deriv_o = factor*dist_xy
    return deriv_x, deriv_y, deriv_o


def create_data(fixed_nodes,floating_nodes,measurement_pairs,measurements):
    '''
        fixed nodes: indices in measurement_pairs corresponding to fixed nodes
        floating nodes: indices in measurement_paris correspond to floating nodes
        measurements pairs: each column in measurements corresponds to a distance between two nodes as specified by the rows in this matrix
        measurements: num_measurements x num pairs observed distances
        
    '''
    num_fixed = fixed_nodes.shape[0]
    num_floating = floating_nodes.shape[0]
    num_measurements = measurements.shape[0]
    measurement_pairs = measurement_pairs.copy()
    
    floating_to_fixed = {} #contains an array of indices in measurement_pairs for each node
    floating_to_floating = {} #similar
    bars = {1:(2,1.37),3:(4,1.37),5:(6,1.37),7:(8,1.37),9:(10,1.37),11:(12,1.37)} #nodes with bar connections: FROM: (TO,LENGTH)
    
    #for each floating node, find the measurement pairs to fixed nodes
    for node_idx, node in enumerate(floating_nodes):
        #get measurement columns corresponding to this node (to/from)        
        node_dist_idx = np.where((measurement_pairs[:,0] == node) | (measurement_pairs[:,1] == node))[0]
        #create new array
        floating_to_fixed[node] = []        
        for i in node_dist_idx:
            if(measurement_pairs[i,0] in fixed_nodes or measurement_pairs[i,1] in fixed_nodes):
                #found one
                floating_to_fixed[node].append(i)
                
    #for each floating node, find the measurement pairs to other floating nodes
    for node_idx, node in enumerate(floating_nodes):
        #get measurement columns corresponding to this node (to/from)        
        node_dist_idx = np.where((measurement_pairs[:,0] == node) | (measurement_pairs[:,1] == node))[0]
        #create new array
        floating_to_floating[node] = []        
        for pair_idx in node_dist_idx:
            #to prevent doubles, we only add pairs originating FROM this node TO another floating node
            if((measurement_pairs[pair_idx,0] == node) and measurement_pairs[pair_idx,1] in floating_nodes):
                #found one
                floating_to_floating[node].append(pair_idx)
    
    
    #Find all the floating node positions with enough measurements for ranging
    #enough means more than 3 valid measurements to fixed nodes
    floating_valid_points = np.zeros((num_measurements,num_floating),dtype=bool)
    measurements_valid = (measurements>1) & (measurements<20) #this matrix contains a one for each valid measurements
    #a one in the above matrix indicates that there are enough valid measurements between a floating node and fixed nodes available for the 
    #columns correspond to indices in floating_nodes
    for node_idx, node in enumerate(floating_nodes):
        for timestep_idx in xrange(num_measurements):
            #get all measurements to fixed nodes from this node
            to_fixed = floating_to_fixed[node]
            valid_cnt = 0
            for pair_idx in to_fixed:
                if(measurements_valid[timestep_idx,pair_idx]):
                    #this is a valid measurement, use it
                    valid_cnt += 1
            if(valid_cnt >= 4):
                #we can estimate this location at this timestep
                floating_valid_points[timestep_idx,node_idx] = True

    #we now know all the valid coordinates to estimate, so we are ready to create constraints
        
    #print floating_to_fixed
    #print floating_to_floating
        
    #create distance constraints between floating nodes and fixed nodes for all valid time steps with valid measurements per floating node
    constraints = []
    
    for timestep_idx in xrange(num_measurements): #iterate over time steps
        for node_idx, node in enumerate(floating_nodes): #iterate over nodes
            if(floating_valid_points[timestep_idx,node_idx]):
                #This is a valid measurement timestep, 
                #add a constraint for each valid measurement to the respective anchor.
                to_fixed = floating_to_fixed[node]
                for pair_idx in to_fixed:
                    if(measurements_valid[timestep_idx,pair_idx]):
                        constraints.append(("float_to_fixed",
                                            measurement_pairs[pair_idx],
                                            measurements[timestep_idx,pair_idx],
                                            timestep_idx))
                #add constraints to other valid floating nodes
                to_floating = floating_to_floating[node]
                for pair_idx in to_floating:
                    if(measurements_valid[timestep_idx,pair_idx]):
                        pair = measurement_pairs[pair_idx]
                        from_ = np.where(floating_nodes == pair[0])[0][0]
                        to_ = np.where(floating_nodes == pair[1])[0][0]
                        if(floating_valid_points[timestep_idx,from_] and floating_valid_points[timestep_idx,to_]): #make sure destination node is valid at the current timestep
                            
                            constraints.append(("float_to_float",
                                            measurement_pairs[pair_idx],
                                            measurements[timestep_idx,pair_idx],
                                            timestep_idx))
                        
                #add bar constraints
                if(node in bars):
                    b = bars[node]
                    bar_node = b[0]
                    #check whether destination node location is valid at this timestep
                    if(floating_valid_points[timestep_idx,np.where(floating_nodes == bar_node)[0][0]]):
                        #ok, we can add the constraint
                        constraints.append(("bar",
                                            [node,bar_node],
                                            b[1],
                                           timestep_idx))
    
    #create variables (this is the fun part)
    variable_to_node = [] #indicates which variable corresponds to which node
    for n in fixed_nodes:
        variable_to_node.append(n) #first indices are fixed nodes
    for timestep_idx in xrange(num_measurements): #iterate over time steps
        for node_idx, node in enumerate(floating_nodes): #iterate over nodes
            if(floating_valid_points[timestep_idx,node_idx]):
                variable_to_node.append((node,timestep_idx)) #we store the floating nodes as node, timestep tuples
    variable_to_node_map = {}
    for v_idx,v in enumerate(variable_to_node):
        variable_to_node_map[v] = v_idx
    
    #print len(variable_to_node)
    
    offset_variable_to_pair = [] #offset variables: pairs of nodes == tricky
    for c in constraints:
        #find out if we already have an offset variable for this constraint pair
        if(not c[0]=="bar"): #bar constraints don't have offsets
            pair = tuple(np.sort(c[1]))
            if(not pair in offset_variable_to_pair):
                offset_variable_to_pair.append(pair)
    
    num_offsets = len(offset_variable_to_pair)
    num_variables = len(variable_to_node)
    num_constraints = len(constraints)
    
    #print len(offset_variable_to_pair)
    #TODO: constraints
    #print len(constraints)
    
    #Generate the measurement vector, variable to constraint matrices and offset to constraint matrices
    measurement_vector = np.zeros(num_constraints)
    variables_to_CSTR_X = np.zeros((num_constraints,num_variables))
    variables_to_CSTR_Y = np.zeros(variables_to_CSTR_X.shape)
    offsets_to_CSTR = np.zeros((num_constraints,num_offsets))
    #print floating_valid_points
    for c_idx,c in enumerate(constraints):
        measurement_vector[c_idx] = c[2]
        #find the correct offset variable
        if(c[0]=="bar"):
            #bar constraints don't have offsets
            offsets_to_CSTR[c_idx,:] = 0 
        else:
            pair = tuple(np.sort(c[1]))
            offset_idx = offset_variable_to_pair.index(pair)
            offsets_to_CSTR[c_idx,offset_idx] = 1 
        #find the correct nodal coordinate variables
        if(c[0]=="bar" or c[0]=="float_to_float"):
            #print c
            from_ = tuple((c[1][0],c[3])) #node,timestep
            to_ = tuple((c[1][1],c[3]))
            from_idx = variable_to_node_map[from_]
            to_idx = variable_to_node_map[to_]
            variables_to_CSTR_X[c_idx,from_idx] = 1
            variables_to_CSTR_Y[c_idx,to_idx] = 1
        else:
            #float to fixed
            if(c[1][0] in fixed_nodes):
                from_ = c[1][0]
                to_ = tuple((c[1][1],c[3]))
            else:
                from_ = tuple((c[1][0],c[3]))
                to_ = c[1][1]

            from_idx = variable_to_node_map[from_]
            to_idx = variable_to_node_map[to_]
            variables_to_CSTR_X[c_idx,from_idx] = 1
            variables_to_CSTR_Y[c_idx,to_idx] = 1
        #variables_to_CSTR_X[c_idx,variable_to_node.index(c[1][0])] = 1
        #variables_to_CSTR_Y[c_idx,variable_to_node.index(c[1][1])] = 1
        
        
    def deriv_func(N,o):
        X = variables_to_CSTR_X.dot(N)
        Y = variables_to_CSTR_Y.dot(N)
        O = offsets_to_CSTR.dot(o.reshape((num_offsets,1))) 
        #print X.shape
        #print Y.shape
        #print O.shape
        #print m.shape
        dX,dY,dO = derivative_cost_function(X,Y,measurement_vector.ravel(),O.ravel())
        dN = variables_to_CSTR_X.T.dot(dX) +  variables_to_CSTR_Y.T.dot(dY)
        dN[0,:] = 0
        dN[1,1:] = 0
        dN[2,2] = 0
        do = offsets_to_CSTR.T.dot(dO.reshape((-1,1))).ravel()
        return dN,do
    
    def cost_func(N,o):
        X = variables_to_CSTR_X.dot(N)
        Y = variables_to_CSTR_Y.dot(N)
        O = offsets_to_CSTR.dot(o.reshape((num_offsets,1))) 
        return cost_function(X,Y,measurement_vector.ravel(),O.ravel())
    
    #print offsets_to_CSTR
    #print variables_to_CSTR_X
    #create m vector
    return deriv_func,cost_func, num_variables, num_offsets, variable_to_node, variable_to_node_map, offset_variable_to_pair

In [7]:
#in case you want to save/load data
#import scipy.io as sio
#sio.savemat("data_outdoors_preprocessed.mat",
#            {'fixed_nodes':fixed_nodes,'floating_nodes':floating_nodes,'dist_pairs':dist_pairs,'m':m})
#d = sio.loadmat('data_outdoors_preprocessed.mat')
#fixed_nodes = d['fixed_nodes']
#floating_nodes = d['floating_nodes']
#dist_pairs = d['dist_pairs']
#meas = d['meas']

In [ ]:
#This is the "main loop"
cf = 10
cf_min = cf
results_and_costs = []
while cf>0.00001:
    fixed_nodes = np.array([16,15,13,14,17,18,19,20],dtype=int) #list the anchors
    floating_nodes = np.array([3,4,5,6,7,8,9,10,],dtype=int)#,2],dtype=int) #list the nodes on the robot
    #dist_pairs = np.array([[2,13],[2,14],[2,15],[2,16],
    #                       [1,13],[1,14],[1,15],[1,16],
    #                       [4,13],[1,14],[1,15],[1,16],
    #                       [6,13],[1,14],[1,15],[1,16],
    #                       [8,13],[1,14],[1,15],[1,16],
    #                       [,13],[1,14],[1,15],[1,16],
    #                       [1,13],[1,14],[1,15],[1,16]
    #                      ],dtype=int)
                           #np.array([[1,10],[2,10],[4,10],[7,10],[8,10],[12,10],[13,10],[16,10]],dtype=int)
    dist_pairs = topics_node_to_node #list the nodal pairs (columns in measurements)

    a = np.arange(meas[0].shape[0]) #which data to use (I like to shuffle and then subsample data)
    np.random.shuffle(a)
    m = meas[0][a[:400],:]#[:,(0,1,2,3,4,5,6,8)]
    
    #create constraints
    df,cf,DOF,num_offsets,variable_to_node,variable_to_node_map, offset_variable_to_pair  = create_data(fixed_nodes,floating_nodes,dist_pairs,m)

    #Initialize optimization variables
    N,o = np.random.randn(DOF,3)*3,np.zeros(num_offsets)+3.8
    N[0] = 0
    N[1,1:] = 0
    N[2,2] = 0     
    #B=np.array([[ 0.,          0.,          0.        ],
    #                 [ -3.38,  0.,          0.        ],
    #                 [ -0.7039,  -2.189,  0.        ]])
    #Set this if you know the first three anchor locations
    B = np.array([[0,0,0],
                 [9.97,0,0],
                 [3.15,11.16,0]])
    #N[:3] = B

    dN,do = df(N,o)
    #print cf(N,o)

    import scipy.optimize as so
    Nf = N.ravel()
    X_initial = np.hstack((Nf,o))
    def cost_fun(x):
        Nt = x[:-o.shape[0]].reshape((DOF,3))
        #Nt[:3] = B #uncomment if the first 3 anchor locations are known
        #Regularization of the offsets
        reg = 0.001*np.sum((x[-o.shape[0]:]-3.8)**2)#+0.5*(x[11]-2.5)**2#+0.05*(x[10]-2.)**2+0.05*(x[9]-2.)**2
        return cf(Nt ,x[-o.shape[0]:])+reg
    def cost_grad(x):
        dN,do = df(x[:-o.shape[0]].reshape((DOF,3)),x[-o.shape[0]:])
        #dN[:3] = 0 #uncomment if the first 3 anchor locations are known
        #do[:] = 0
        #Regularization of the offsets
        do += 0.001*2*(x[-o.shape[0]:]-3.8)
        dN = dN.ravel()
        return np.hstack((dN,do))


    resall = []
    def progress(x):
        resall.append(x)

    #Truncated Newton is fast, but BFGS is also an OK choice
    res = so.minimize(cost_fun,X_initial,jac=cost_grad,method='TNC')#,options={'xtol':1e-15,'disp':True})
    #res = so.fm
    #print df(res[:-o.shape[0]].reshape((DOF,3)),res[-o.shape[0]:])[1]
    cf = res.fun#cost_fun(res.x)
    results_and_costs.append((res.copy(),cf,variable_to_node_map,offset_variable_to_pair,a.copy()))
    if(cf<cf_min):
        res_min = res.copy()
        DOF_min = DOF
        variable_to_node_map_min =  variable_to_node_map
        offset_variable_to_pair_min = offset_variable_to_pair
        cf_min = cf
    print np.sqrt(cf)
    

0.214538493577
0.203845893025
0.200950066441
0.163409803814
0.248281463984
0.202402865018
0.166095130279
0.161378311887
0.254283987108
0.189255301955
0.267274160651
0.218182194274
0.0833889857914
0.227151070085
0.196155471743
0.272930021357
0.182951805017
0.264037549314
0.237393023686
0.183803613417
0.17112068895
0.254473038117
0.285368673382
0.328379212891
0.0657679199866
0.194747174228
0.216636156032
0.202550515725
0.202275764077
0.243026067662
0.131094890078
0.244348763887
0.302228172415
0.192607048038
0.103762872544
0.277637902502
0.198257947901
0.242733720411
0.270135304423
0.206688096044
0.161080398161
0.246464238035
0.192326972335
0.233455083151
0.277352891701
0.282359923716
0.180903569784
0.165084480046
0.172489941354
0.0789478117531
0.221212012911
0.297485099405
0.173030145745
0.187650559582
0.343949772721
0.196975432062
0.21627599129
0.158639099652
0.192690395166
0.0994792567568
0.142166572121
0.161332888249
0.105138970074
0.129398425912
0.243782004672
0.260536929404
0.140941

In [126]:
(res_min['fun'])
#a.shape

0.0020906791517911532

In [128]:
#cost_fun(res_min)
res=res_min['x'].copy()
cf_min

0.0020906791517911532

In [130]:
#3D plot of the result
print res[-o.shape[0]:].round(2)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p = res[:fixed_nodes.shape[0]*3].reshape((fixed_nodes.shape[0],3))[:fixed_nodes.shape[0]]
#p[0,1]+=1
p2 = res[:-len(offset_variable_to_pair_min)].reshape(((res.shape[0]-len(offset_variable_to_pair_min))/3,3))[fixed_nodes.shape[0]:]
ax.scatter(p[:,0],p[:,1],p[:,2],c='b')
for i in xrange(fixed_nodes.shape[0]):
    ax.text(p[i,0],p[i,1],p[i,2],fixed_nodes[i])

ax.scatter(B[:,0],B[:,1],B[:,2],c='g')
ax.scatter(p2[:,0],p2[:,1],p2[:,2],c='y')
#ax.scatter(pc[:,0],pc[:,1],pc[:,2],c='b')
#ax.scatter(p2c[:,0],p2c[:,1],p2c[:,2],c='r')
ax.set_aspect('equal')

[-7.25  0.28  0.79 -6.7   1.82  2.04 -5.81  1.83  3.73  3.89  3.85  3.88
  4.02  3.76  3.93  3.89  3.75  3.89  3.81  3.94  3.79  3.75  3.4   3.85
  3.71  3.72  3.82  3.92  3.73  3.84  3.83  3.84  3.73  4.09  3.78  3.67
  3.86  3.72  3.81  3.75  3.69  3.85  3.8   3.77  3.87  3.87  3.87  3.86
  3.7   3.82  4.03  3.82  3.67  3.75  3.86  3.77  3.74  3.85  3.42  3.75
  3.85  3.79  3.87]


In [101]:
#Coordinates of first 8 nodes (==anchors)
p_all = res[:24].reshape((8,3))
print p_all
print B
#np.linalg.norm(p_all[20]-p_all[21])

[[  0.           0.           0.        ]
 [ 10.11481398   0.           0.        ]
 [  3.52651551  10.95072281   0.        ]
 [  6.5036523    8.27974315   4.98545859]
 [ 13.03070808  10.72102472   0.37880569]
 [ 16.12345458   1.66013333   0.67121005]
 [ 16.41303364   5.22995144   0.52341973]
 [  0.34356977   5.78397216   0.20109945]]
[[  0.     0.     0.  ]
 [  9.97   0.     0.  ]
 [  3.15  11.16   0.  ]]


In [103]:
#average offset
res[-len(offset_variable_to_pair_min):].mean()

3.762853608096266

In [28]:
#print offsets
res[-len(offset_variable_to_pair_min):]


array([ 4.16799159,  3.63648985,  4.03304079,  3.83795675,  4.02615586,
        3.68629254,  4.03762013,  3.89848137,  4.18677463,  3.62661513,
        4.02919902,  3.79128936,  3.24252253,  3.74993813,  3.54417684,
        3.63720535,  4.21208742,  3.54548237,  4.18295844,  3.71217001,
        4.06504594,  3.65841194,  4.2756098 ,  3.93173312,  3.3867485 ,
        3.76147812,  3.18438827,  3.92609695])

In [19]:
#save the offsets if needed
import scipy.io as sio
#sio.savemat("offsets_outdoors_nobasepoints_given_longlongrun.mat",{"offset_variable_to_pair":np.array(offset_variable_to_pair_min),'offsets':res_min[-len(offset_variable_to_pair_min):]})
